notebook used to run & test ddpm_conditional.py functions

In [1]:

#? resources
#? vid 1: https://www.youtube.com/watch?v=HoKDTa5jHvg&t=177s
# explains how ddpm works
#? vid2: https://www.youtube.com/watch?v=TBCRlnwJtZU
# goes over implementation to ddpm

import os
import copy
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
from helper_utils import *

from modules import UNet, UNet_conditional, EMA
import logging 
from torch.utils.tensorboard import SummaryWriter
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

# library for quantization
try:
    import bitsandbytes as bnb
    print('imported bitsandbytes')
    
except:
    print('cant import bitsandbytes')
    bnb = None

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=256, device="cuda", USE_GPU = True):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size #resoultion of image #: side note from video --> for higher resolutions, training seperate upsamplers instead of training on bigger resolution images
        self.device = device
        
        if USE_GPU and torch.cuda.is_available():
            print("CUDAAAAAAAAAAA")
            self.use_cuda = torch.device('cuda')
        else:
            self.use_cuda = torch.device('cpu')
        
        #? right now using simple beta schedule --> open AI using cosine scheduler        
        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)
        
        #! try implementing cosine scheduler
        
    def prepare_noise_schedule(self):
        #? Creates a one-dimensional tensor of size steps whose values are evenly spaced from start to end, inclusive
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)
    
    def noise_images(self, x, t):
        """Adds noise to image. You can iteratively add noise to image but vid 1 showed 
        a simplification that adds noise in 1 step. Which is this implementation
        Args:
            x (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: returns image with noise added on
        """
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        E = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * E, E
    
    def sample_timesteps(self, n):
        """_summary_

        Args:
            n (_type_): _description_

        Returns:
            _type_: _description_
        """
        #? needed for algorithm for training
        return torch.randint(low=1, high=self.noise_steps, size=(n,))
    

    def sample(self, model, n, labels, channels=3, cfg_scale=3):
        """implements algorithm 2 from the ddpm paper in vid 1

        Args:
            model (_type_): _description_
            n (int): number of images we want to sample 

        Returns:
            _type_: _description_
        """
        logging.info(f"Sampling {n} new images....")
        #? see here for why we set model.eval() https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
        #? essentially disables some some parts of torch for specific steps
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            #? create initial images by sampling over normal dist (step 1)
            x = torch.randn((n, channels, self.img_size, self.img_size)).to(self.device)
            
            #? step 2, 3, 4
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device) #? tensor of timestep
                predicted_noise = model(x, t, labels) #? feed that into model w/ current images
                
                #? noise
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                
                #? only want noise for timestemps greater than 1. done so b/c in last iteration, would make final outcome worse due to adding noise to finalized pixels
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                    
                #? alter image by removed a little bit of noise
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        
        #? switch back to train    
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2 #? brings back value to 0-1 range 
        x = (x * 255).type(torch.uint8) #? bring back values to pixel range for viewing image
        return x

<module 'torch._C' from 'c:\\Users\\Efran\\anaconda3\\envs\\compsci682\\lib\\site-packages\\torch\\_C.cp310-win_amd64.pyd'>
False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/Efran/anaconda3/envs/compsci682/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capab

c:\Users\Efran\anaconda3\envs\compsci682\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
c:\Users\Efran\anaconda3\envs\compsci682\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: C:\Users\Efran\anaconda3\envs\compsci682 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)



===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/Efran/anaconda3/envs/compsci682/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.9.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/

### test ddpm + conditional + ema

In [19]:
import argparse
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.run_name = "base_mnist_ddpm_conditional_ema"
args.epochs = 50
args.batch_size = 10


args.dataset_path = r"mnist"
args.image_size = 56
args.num_classes = 10
args.channels = 1

args.optim = ['quant', 'prune', 'deep']
args.optim = ['deep']
args.optim = []
args.groups= [1, 2, 2, 2, 2, 2, 2, 2, 1]
# [1, 2, 2, 2, 2, 2, 2, 2, 1]
# [1, 4, 4, 4, 4, 4, 4, 4, 1]
# [1,32,64,128,128,64,32,1]

args.device = "cuda"
args.lr = 3e-4

In [21]:
setup_logging(args.run_name)
device = args.device

if not args.optim:
    print('using no optimization methods')

if 'deep' in args.optim:
    print('using depthwise convolution optmization')
    groups = args.groups
else:
    print('not using depthwise convolution optmization')
    groups = [1, 1, 1, 1, 1, 1, 1, 1, 1]

model = UNet_conditional(c_in=args.channels, c_out=args.channels, size=args.image_size, time_dim=args.image_size*4, num_classes=args.num_classes, device=args.device, args=args, groups=groups).to(device)

if 'quant' in args.optim:
    print('using quantization optimizer')
    optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=args.lr)
else:
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    
mse = nn.MSELoss()
diffusion = Diffusion(img_size=args.image_size, device=device)
ema = EMA(0.995)
ema_model = copy.deepcopy(model).eval().requires_grad_(False)
print('initialized model!')

logger = SummaryWriter(os.path.join("runs", args.run_name))
dataloader = get_data(args)
l = len(dataloader)
print(f"Size of his data set after applying {args.batch_size} data points per batch:", len(dataloader))

l = len(dataloader)

using no optimization methods
not using depthwise convolution optmization
CUDAAAAAAAAAAA
using default groups
using these groups: [1, 1, 1, 1, 1, 1, 1, 1, 1]
CUDAAAAAAAAAAA
initialized model!
using mnist dataset
torch.Size([60000, 28, 28])
loaded mnist training dataset
Size of his data set:  60000
Size of his data set after applying 10 data points per batch: 6000


In [22]:
print('starting loop')
for epoch in range(args.epochs):
    logging.info(f"Starting epoch {epoch}:")
    #? alorithm 1 from vid 1  
    pbar = tqdm(dataloader)
    for i, (images, labels) in enumerate(pbar):
    #for i, (images, _) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        
        if np.random.random() < 0.1:
            labels = None
            
        predicted_noise = model(x_t, t, labels)
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema.step_ema(ema_model, model)

        pbar.set_postfix(MSE=loss.item())
        logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

    if epoch == 0 or (epoch + 1) % 10 == 0:
        print('generating sample images')
        labels = torch.arange(10).long().to(device)
        sampled_images = diffusion.sample(model, n=len(labels), channels=args.channels, labels=labels)
        ema_sampled_images = diffusion.sample(ema_model, n=len(labels), channels=args.channels, labels=labels)

        print('saving sample images & models')
        sample_images_path = os.path.join("results", args.run_name)
        sample_image_name = f"{epoch}_.jpg"

        save_images(sampled_images, sample_images_path, sample_image_name, epoch=epoch)
        save_images(ema_sampled_images, sample_images_path, 'ema' + sample_image_name, epoch=epoch)

        torch.save(model, os.path.join("models", args.run_name, f"{epoch}_ckpt.pt"))
        torch.save(ema_model, os.path.join("models", args.run_name, f"{epoch}_ema_ckpt.pt"))
        torch.save(optimizer, os.path.join("models", args.run_name, f"{epoch}_optim.pt"))

04:33:06 - INFO: Starting epoch 0:


starting loop


  0%|          | 11/6000 [00:01<13:53,  7.18it/s, MSE=0.505]


KeyboardInterrupt: 

### ddpm + conditional + ema + deepwise convolution

In [2]:
import argparse
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.run_name = "d_2_mnist_ddpm_conditional_ema"
args.epochs = 50
args.batch_size = 16


args.dataset_path = r"mnist"
args.image_size = 56
args.num_classes = 10
args.channels = 1

args.optim = ['quant', 'prune', 'deep']
args.optim = ['deep']
#args.optim = []
args.groups= [1, 2, 2, 2, 2, 2, 2, 2, 1]
# [1, 2, 2, 2, 2, 2, 2, 2, 1]
# [1, 4, 4, 4, 4, 4, 4, 4, 1]
# [1,32,64,128,128,64,32,1]

args.device = "cuda"
args.lr = 3e-4

In [3]:
setup_logging(args.run_name)
device = args.device

if not args.optim:
    print('using no optimization methods')

if 'deep' in args.optim:
    print('using depthwise convolution optmization')
    groups = args.groups
else:
    print('not using depthwise convolution optmization')
    groups = [1, 1, 1, 1, 1, 1, 1, 1, 1]

model = UNet_conditional(c_in=args.channels, c_out=args.channels, size=args.image_size, time_dim=args.image_size*4, num_classes=args.num_classes, device=args.device, args=args, groups=groups).to(device)

if 'quant' in args.optim:
    print('using quantization optimizer')
    optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=args.lr)
else:
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    
mse = nn.MSELoss()
diffusion = Diffusion(img_size=args.image_size, device=device)
ema = EMA(0.995)
ema_model = copy.deepcopy(model).eval().requires_grad_(False)
print('initialized model!')

logger = SummaryWriter(os.path.join("runs", args.run_name))
dataloader = get_data(args)
l = len(dataloader)
print(f"Size of his data set after applying {args.batch_size} data points per batch:", len(dataloader))

l = len(dataloader)

using depthwise convolution optmization
CUDAAAAAAAAAAA
using deepwise convolution optimization Unet
using these groups: [1, 2, 2, 2, 2, 2, 2, 2, 1]
CUDAAAAAAAAAAA
initialized model!
using mnist dataset
torch.Size([60000, 28, 28])
loaded mnist training dataset
Size of his data set:  60000
Size of his data set after applying 16 data points per batch: 3750


In [4]:
print('starting loop')
for epoch in range(args.epochs):
    logging.info(f"Starting epoch {epoch}:")
    #? alorithm 1 from vid 1  
    pbar = tqdm(dataloader)
    for i, (images, labels) in enumerate(pbar):
    #for i, (images, _) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        
        if np.random.random() < 0.1:
            labels = None
            
        predicted_noise = model(x_t, t, labels)
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema.step_ema(ema_model, model)

        pbar.set_postfix(MSE=loss.item())
        logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

    if epoch == 0 or (epoch + 1) % 10 == 0:
        print('generating sample images')
        labels = torch.arange(10).long().to(device)
        sampled_images = diffusion.sample(model, n=len(labels), channels=args.channels, labels=labels)
        ema_sampled_images = diffusion.sample(ema_model, n=len(labels), channels=args.channels, labels=labels)

        print('saving sample images & models')
        sample_images_path = os.path.join("results", args.run_name)
        sample_image_name = f"{epoch}_.jpg"

        save_images(sampled_images, sample_images_path, sample_image_name, epoch=epoch)
        save_images(ema_sampled_images, sample_images_path, 'ema' + sample_image_name, epoch=epoch)

        torch.save(model, os.path.join("models", args.run_name, f"{epoch}_ckpt.pt"))
        torch.save(ema_model, os.path.join("models", args.run_name, f"{epoch}_ema_ckpt.pt"))
        torch.save(optimizer, os.path.join("models", args.run_name, f"{epoch}_optim.pt"))

04:38:29 - INFO: Starting epoch 0:


starting loop


100%|██████████| 3750/3750 [08:22<00:00,  7.46it/s, MSE=0.0107] 
04:46:52 - INFO: Sampling 10 new images....
04:46:52 - INFO: Sampling 10 new images....


generating sample images


999it [01:00, 16.65it/s]
04:47:52 - INFO: Sampling 10 new images....
04:47:52 - INFO: Sampling 10 new images....
999it [00:59, 16.87it/s]


...

saving sample images & models
saved images to epoch folder
saved images to epoch grid image
saved images to epoch folder
saved images to epoch grid image


10:29:00 - INFO: Starting epoch 40:
100%|██████████| 3750/3750 [08:25<00:00,  7.42it/s, MSE=0.00522]
10:37:26 - INFO: Starting epoch 41:
100%|██████████| 3750/3750 [08:25<00:00,  7.42it/s, MSE=0.00755]
10:45:51 - INFO: Starting epoch 42:
100%|██████████| 3750/3750 [08:26<00:00,  7.41it/s, MSE=0.00689]
10:54:17 - INFO: Starting epoch 43:
100%|██████████| 3750/3750 [08:13<00:00,  7.59it/s, MSE=0.00632]
11:02:31 - INFO: Starting epoch 44:
100%|██████████| 3750/3750 [07:57<00:00,  7.85it/s, MSE=0.0136] 
11:10:28 - INFO: Starting epoch 45:
100%|██████████| 3750/3750 [08:07<00:00,  7.69it/s, MSE=0.00531]
11:18:36 - INFO: Starting epoch 46:
100%|██████████| 3750/3750 [08:19<00:00,  7.51it/s, MSE=0.00323]
11:26:55 - INFO: Starting epoch 47:
100%|██████████| 3750/3750 [08:27<00:00,  7.39it/s, MSE=0.0042] 
11:35:22 - INFO: Starting epoch 48:
100%|██████████| 3750/3750 [08:25<00:00,  7.41it/s, MSE=0.00653]
11:43:48 - INFO: Starting epoch 49:
100%|██████████| 3750/3750 [08:23<00:00,  7.45it/s, MSE

generating sample images


999it [00:58, 17.07it/s]
11:53:10 - INFO: Sampling 10 new images....
11:53:10 - INFO: Sampling 10 new images....
999it [00:57, 17.48it/s]


saving sample images & models
saved images to epoch folder
saved images to epoch grid image
saved images to epoch folder
saved images to epoch grid image


### load saved models

In [22]:
loaded_unet_model = torch.load(r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\base_mnist_ddpm_conditional_ema\49_ckpt.pt')
loaded_unet_model.eval()

loaded_ema_model = torch.load(r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\base_mnist_ddpm_conditional_ema\49_ema_ckpt.pt')
loaded_ema_model.eval()

epoch = 49
print('loaded', epoch)


loaded 49


In [23]:
args = {}
args['image_size'] = 56
args['channels'] = 1
args['device'] = "cuda"
device = args['device']

args['name'] = f'base_mnist_ddpm_conditional_ema_{epoch}'
loaded_diffusion = Diffusion(img_size=args['image_size'] , device=device)

CUDAAAAAAAAAAA


In [24]:
labels = torch.arange(10).long().to(device)
sampled_images = diffusion.sample(loaded_unet_model, n=len(labels), channels=args['channels'], labels=labels)
ema_sampled_images = diffusion.sample(loaded_ema_model, n=len(labels), channels=args['channels'], labels=labels)

12:24:03 - INFO: Sampling 10 new images....
12:24:03 - INFO: Sampling 10 new images....
999it [01:00, 16.51it/s]
12:25:04 - INFO: Sampling 10 new images....
12:25:04 - INFO: Sampling 10 new images....
999it [00:59, 16.84it/s]


In [29]:
sample_images_path = os.path.join(r"F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\FINAL", args['name'], 'ddpm_conditional')
sample_image_name = f"{epoch}_.jpg"
print(sample_images_path, sample_image_name)
save_images(sampled_images, sample_images_path, sample_image_name, epoch=epoch)

sample_images_path = os.path.join(r"F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\FINAL", args['name'], 'ddpm_conditional_ema')
save_images(ema_sampled_images, sample_images_path, 'ema' + sample_image_name, epoch=epoch)

F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\FINAL\base_mnist_ddpm_conditional_ema_49\ddpm_conditional 49_.jpg
saved images to epoch folder
saved images to epoch grid image
saved images to epoch folder
saved images to epoch grid image


In [30]:
loaded_unet_model = torch.load(r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\base_mnist_ddpm_conditional_ema\49_ckpt.pt')


In [34]:
print(len(list(loaded_unet_model.named_parameters())))

183


In [43]:
print(list(loaded_unet_model.named_parameters()))

[('inc.double_conv.0.weight', Parameter containing:
tensor([[[[ 9.6185e-02, -6.3882e-02,  8.9101e-02],
          [ 8.4583e-02, -4.2012e-01,  6.0567e-02],
          [-3.9633e-02,  2.3848e-02,  1.2772e-01]]],


        [[[-1.5469e-01, -1.0096e-01, -1.3695e-01],
          [ 2.7466e-01,  2.6574e-01, -1.3480e-02],
          [-4.3361e-02,  1.0525e-01, -2.1086e-01]]],


        [[[-3.3152e-02,  6.9398e-02,  1.6502e-01],
          [ 1.9787e-02,  3.3941e-02,  2.1190e-01],
          [-1.9001e-01,  6.7462e-02,  5.0489e-02]]],


        [[[-4.7834e-02, -5.4754e-02, -1.0464e-01],
          [-1.1624e-01, -4.8877e-02,  4.1481e-02],
          [-7.6110e-02, -9.5583e-02, -6.7954e-02]]],


        [[[ 6.4058e-02,  1.9400e-02, -5.1735e-02],
          [ 1.6596e-01, -3.9879e-02, -1.2562e-01],
          [ 1.7598e-01,  5.6009e-02, -6.6067e-02]]],


        [[[-2.8187e-02,  1.6870e-02,  4.6798e-02],
          [-1.0285e-02, -2.7486e-01, -1.4922e-01],
          [ 2.8812e-02, -1.7561e-01,  1.1676e-01]]],


      

In [33]:
print(len(list(loaded_unet_model.named_buffers())))

0


In [35]:
import torch.nn.utils.prune as prune


In [48]:
loaded_unet_model.inc.double_conv[0].weight

Parameter containing:
tensor([[[[ 9.6185e-02, -6.3882e-02,  8.9101e-02],
          [ 8.4583e-02, -4.2012e-01,  6.0567e-02],
          [-3.9633e-02,  2.3848e-02,  1.2772e-01]]],


        [[[-1.5469e-01, -1.0096e-01, -1.3695e-01],
          [ 2.7466e-01,  2.6574e-01, -1.3480e-02],
          [-4.3361e-02,  1.0525e-01, -2.1086e-01]]],


        [[[-3.3152e-02,  6.9398e-02,  1.6502e-01],
          [ 1.9787e-02,  3.3941e-02,  2.1190e-01],
          [-1.9001e-01,  6.7462e-02,  5.0489e-02]]],


        [[[-4.7834e-02, -5.4754e-02, -1.0464e-01],
          [-1.1624e-01, -4.8877e-02,  4.1481e-02],
          [-7.6110e-02, -9.5583e-02, -6.7954e-02]]],


        [[[ 6.4058e-02,  1.9400e-02, -5.1735e-02],
          [ 1.6596e-01, -3.9879e-02, -1.2562e-01],
          [ 1.7598e-01,  5.6009e-02, -6.6067e-02]]],


        [[[-2.8187e-02,  1.6870e-02,  4.6798e-02],
          [-1.0285e-02, -2.7486e-01, -1.4922e-01],
          [ 2.8812e-02, -1.7561e-01,  1.1676e-01]]],


        [[[-1.2635e-01,  1.8692e-02,

In [49]:
prune.random_unstructured(loaded_unet_model.inc.double_conv[0], name='weight', amount=0.3)

Conv2d(1, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [51]:
print(dict(loaded_unet_model.named_buffers()).keys())

dict_keys(['inc.double_conv.0.weight_mask'])


In [ ]:
loaded_unet_model